# Toxic Comment Classifier DNN 

This notebook will focus on the use of Deep Neural Networks to tackle the problem of tox comment classification. Starting from the work done in the `toxic-comment-classifier-classical-model.ipynb

In [89]:
import pandas as pd 
from  sklearn.model_selection import train_test_split

data = pd.read_csv('./data/train.csv')
print(data.shape)
X = data['comment_text']
y = data[data.columns[2:]].values

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.2, random_state=42)

(159571, 8)


In [90]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(X_train)
tokenized_train = tokenizer.texts_to_sequences(X_train)
tokenized_test = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index


In [91]:
print(len(word_index))
X_train.shape

183197


(127656,)

We're going to use a pretrained Word2Vector as the basis of our embedded vocabulary. The pretrained Word2Vec model is going to be Facebook's [fastText](https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec). For more info please find it [here](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md).

In [77]:
import numpy as np
embedding_dim = 300

def process_pretrained_word_vec(line):
    values = line.rstrip().rsplit(' ', embedding_dim)
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    return (word, coefs)
    
with open('./wiki.en.vec', encoding='utf8') as f:
#     [line[0] for line.rstrip().rsplit(' ', embedding_dim) in f]
    embedding = dict(map(process_pretrained_word_vec, f))


In [78]:
vect  = embedding['paris'] - embedding['france'] + embedding['italy']
np.array_equal(vect, embedding['rome'])
# [k for k, v in embedding.items() if np.array_equal(v, vect)]

False

In [79]:
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [80]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.065334  , -0.093031  , -0.017571  , ...,  0.16642   ,
        -0.13079   ,  0.035397  ],
       [-0.21341   ,  0.15353   ,  0.05288   , ..., -0.025937  ,
        -0.072507  ,  0.14989001],
       ...,
       [-0.25375   , -0.24808   , -0.17106   , ...,  0.28101999,
         0.30978999,  0.233     ],
       [ 0.083979  , -0.12548999,  0.038038  , ..., -0.10317   ,
         0.12037   ,  0.18133   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## CNN 

In [87]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing.sequence import TimeseriesGenerator

max_len = 70
model = Sequential()

model.add(Embedding(44893,
                    embedding_dim, weights=[embedding_matrix],
                    input_length=max_len, trainable=True)
         )

# Add Convolutional layer
model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
model.add(MaxPooling1D(3))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
# Add fully connected layers
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6, activation='sigmoid'))

ResourceExhaustedError: OOM when allocating tensor with shape[44893,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: embedding_11/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4018753, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_11/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'embedding_11/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658,